In [20]:
import os
from dotenv import load_dotenv

from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
    
)

from langchain.chat_models import ChatOpenAI
from tqdm.notebook import tqdm

import json
import pickle
import string

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import tiktoken

from pdb import set_trace

# Load OPENAI_API_KEY from .env file
load_dotenv()

ModuleNotFoundError: No module named 'langchain'

In [14]:
PRICE_PER_1K_TOKENS_PROMPT = 0.0015
PRICE_PER_1K_TOKENS_COMPLETE = 0.002

MODEL_ID = "gpt-3.5-turbo-0613"

def num_tokens_from_string(string):
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model(MODEL_ID)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [7]:
tasks = [
    'wdcproducts-80cc-seen-sampled-250-gs-2_domain-simple-force-related-3',
    'wdcproducts-80cc-seen-sampled-250-gs-2_domain-simple-force-random-3',
    'wdcproducts-80cc-seen-sampled-250-gs-2_domain-simple-force-handpicked-3',
    'wdcproducts-80cc-seen-sampled-250-gs-2_domain-simple-force-related-5',
    'wdcproducts-80cc-seen-sampled-250-gs-2_domain-simple-force-random-5',
    'wdcproducts-80cc-seen-sampled-250-gs-2_domain-simple-force-handpicked-5',
    
    'abt-buy-sampled-gs_domain-simple-force-related-3',
    'abt-buy-sampled-gs_domain-simple-force-random-3',
    'abt-buy-sampled-gs_domain-simple-force-handpicked-3',
    'abt-buy-sampled-gs_domain-simple-force-related-5',
    'abt-buy-sampled-gs_domain-simple-force-random-5',
    'abt-buy-sampled-gs_domain-simple-force-handpicked-5',
    
    'amazon-google-sampled-gs_domain-simple-force-related-3',
    'amazon-google-sampled-gs_domain-simple-force-random-3',
    'amazon-google-sampled-gs_domain-simple-force-handpicked-3',
    'amazon-google-sampled-gs_domain-simple-force-related-5',
    'amazon-google-sampled-gs_domain-simple-force-random-5',
    'amazon-google-sampled-gs_domain-simple-force-handpicked-5',
    
    'walmart-amazon-sampled-gs_domain-simple-force-related-3',
    'walmart-amazon-sampled-gs_domain-simple-force-random-3',
    'walmart-amazon-sampled-gs_domain-simple-force-handpicked-3',
    'walmart-amazon-sampled-gs_domain-simple-force-related-5',
    'walmart-amazon-sampled-gs_domain-simple-force-random-5',
    'walmart-amazon-sampled-gs_domain-simple-force-handpicked-5',
    
    'dblp-scholar-sampled-gs_domain-simple-force-related-3',
    'dblp-scholar-sampled-gs_domain-simple-force-random-3',
    'dblp-scholar-sampled-gs_domain-simple-force-handpicked-3',
    'dblp-scholar-sampled-gs_domain-simple-force-related-5',
    'dblp-scholar-sampled-gs_domain-simple-force-random-5',
    'dblp-scholar-sampled-gs_domain-simple-force-handpicked-5',
    
    'dblp-acm-sampled-gs_domain-simple-force-related-3',
    'dblp-acm-sampled-gs_domain-simple-force-random-3',
    'dblp-acm-sampled-gs_domain-simple-force-handpicked-3',
    'dblp-acm-sampled-gs_domain-simple-force-related-5',
    'dblp-acm-sampled-gs_domain-simple-force-random-5',
    'dblp-acm-sampled-gs_domain-simple-force-handpicked-5',
    ]

In [8]:
attributes = ['default']

for task in tasks:
    for attribute in attributes:
        
        token_count_prompt = 0
        token_count_completion = 0
        
        # open the JSON file in read mode
        with open(f'../tasks/{task}.json', 'r') as f:
            # load the JSON data from the file and convert it into a dictionary
            task_dict = json.load(f)

        human_template = """{task_prefix}{input_string}"""
        human_prompt = PromptTemplate(
                template=human_template,
                input_variables=['task_prefix', 'input_string']
        )

        model = ChatOpenAI(model_name=MODEL_ID, temperature=0)
        
        preds = []
        prompts = []
        
        for example in tqdm(task_dict['examples']):
            
            context_prompts = []
            
            if 'rules' in task:
                prefix = task_dict['task_prefix']
                context_prompts.append(prefix)
            
            inputs_split = example['input'].split('\n\n')
            
            matches = inputs_split[0].split('\n')[1:]
            non_matches = inputs_split[1].split('\n')[1:]
            task_full = inputs_split[2]
            question = task_full.split('\n')[0]

            for match in matches:
                match = match.replace('{', '').replace('}', '')
                
                example_human = HumanMessagePromptTemplate.from_template(f'{question}\n{match}')
                context_prompts.append(example_human)
                example_ai = AIMessagePromptTemplate.from_template(f'Yes.')
                context_prompts.append(example_ai)
            
            for non_match in non_matches:
                non_match = non_match.replace('{', '').replace('}', '')
                
                example_human = HumanMessagePromptTemplate.from_template(f'{question}\n{non_match}')
                context_prompts.append(example_human)
                example_ai = AIMessagePromptTemplate.from_template(f'No.')
                context_prompts.append(example_ai)        

            human_message_prompt = HumanMessagePromptTemplate.from_template(f'{task_full}')

            context_prompts.append(human_message_prompt)

            chat_prompt = ChatPromptTemplate.from_messages(context_prompts)
            prompts.append(chat_prompt.format())
            tokens = num_tokens_from_string(chat_prompt.format())
            token_count_prompt += tokens
            
            llm_chain = LLMChain(
                prompt=chat_prompt,
                llm=model
            )

            pred = llm_chain.run({})
            preds.append(pred)
            
        targets = [example['target_scores'] for example in task_dict['examples']]

        # save the prompts
        with open(f'../prompts/{task}_{attribute}_{MODEL_ID}_run-1.pickle', 'wb') as handle:
            pickle.dump(prompts, handle, protocol=pickle.HIGHEST_PROTOCOL)
        # save the answers
        with open(f'../answers/{task}_{attribute}_{MODEL_ID}_run-1.pickle', 'wb') as handle:
            pickle.dump(preds, handle, protocol=pickle.HIGHEST_PROTOCOL)

        for pred in preds:
            tokens = num_tokens_from_string(pred)
            token_count_completion += tokens
            
        # Do some data wrangling to format target and preds to match squad V2
        predictions = []
        truth = []
        unclear_answers = 0
        num_long_answers = 0
        for i in range(len(targets)):
            if targets[i]['Yes'] == 1:
                truth.append(1)
            else:
                truth.append(0)
                    
            # handle yes/no answers

            processed_pred = preds[i].strip().translate(str.maketrans('', '', string.punctuation)).lower()

            if processed_pred != 'yes' and processed_pred != 'no':
                print(f'Overlong Answer: {processed_pred}')
                num_long_answers += 1
            if 'yes' in processed_pred:
                processed_pred = 'yes'
            elif 'no' in processed_pred:
                processed_pred = 'no'
            else:
                processed_pred = 'no'
                unclear_answers += 1

            if processed_pred == 'yes':
                predictions.append(1)
            elif processed_pred == 'no':
                predictions.append(0)

        precision = precision_score(truth, predictions)
        recall = recall_score(truth, predictions)
        f1 = f1_score(truth, predictions)
        accuracy = accuracy_score(truth, predictions)
        
        price = (token_count_prompt/1000)*PRICE_PER_1K_TOKENS_PROMPT + (token_count_completion/1000)*PRICE_PER_1K_TOKENS_COMPLETE

        results = {
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1': f1,
            'cost': price
        }

        with open(f"../results/result_{task}_{attribute}_{MODEL_ID}_run-1.json", "w") as outfile:
            json.dump(results, outfile, indent=2)

NameError: name 'json' is not defined